<a href="https://cognitiveclass.ai"><img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/IDSNlogo.png" width = 400> </a>
​
<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>
​

In [5]:
!pip install bs4
!pip install lxml
!pip install html5lib
from bs4 import BeautifulSoup 
import lxml
import html5lib
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

# All requested packages already installed.



#### First we load the data and prepare the dataframe

In [44]:
# web scraping using the BeautifulSoup package
my_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(my_url,'html5lib')
table = soup.find('table_class',{'class':'navbox'})
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
Toronto_df=pd.DataFrame(table_contents)
Toronto_df['Borough']=Toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

# dispaying the dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
Toronto_df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Now verify the data size

In [45]:
Toronto_df.shape

(103, 3)

Load coordinates data and join

In [46]:
coord = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv', index_col='Postal Code')
Toronto_df = Toronto_df.join(coord, on ='PostalCode')
Toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Create a map of Toronto with neighborhoods superimposed on top

In [59]:
#get Toronto coordinates
geolocator = Nominatim(user_agent="myexplorer")
location = geolocator.geocode('Toronto, ON')
loc_Toronto = [location.latitude, location.longitude]
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=loc_Toronto, zoom_start=10)

# add markers to map
for borough, neighborhood, lat, lon in zip(Toronto_df['Borough'], Toronto_df['Neighborhood'], Toronto_df['Latitude'], Toronto_df['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Let's see which Toronto districts have the most neighborhoods

In [60]:
for borough in Toronto_df['Borough'].unique():
    neighs = Toronto_df[Toronto_data['Borough'] == borough].shape[0]
    print("{}: {}".format(borough, neighs))

North York: 24
Downtown Toronto: 17
Queen's Park: 1
Etobicoke: 11
Scarborough: 17
East York: 4
York: 5
East Toronto: 4
West Toronto: 6
East York/East Toronto: 1
Central Toronto: 9
Mississauga: 1
Downtown Toronto Stn A: 1
Etobicoke Northwest: 1
East Toronto Business: 1


North York is the district with more neighbours, let's see the dataframe

In [61]:
NY_df = Toronto_df[Toronto_data['Borough'] == 'North York']
NY_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M3B,North York,Don Mills North,43.745906,-79.352188
10,M6B,North York,Glencairn,43.709577,-79.445073


And create the map

In [65]:
map_Toronto = folium.Map(location=loc_Toronto, zoom_start=10)

# add markers to map
for neighborhood, lat, lon in zip(NY_df['Neighborhood'], NY_df['Latitude'], NY_df['Longitude']):
    label = '{}, North York'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [ ]:
Let's get the top 100 venues that are in Parkwoods within a radius of 500 meters

In [67]:
neigh = NY_df['Neighborhood'].iloc[0]
lat = NY_df['Latitude'].iloc[0]
lon = NY_df['Longitude'].iloc[0]
print('Coordinates of {}: {} {}'.format(neigh, lat, lon))
# type your answer here
CLIENT_ID = '4JW2V1AXWJXREM12VKKJCM4B50FZ1I4TYRS2URMC1WMZRTNM' # your Foursquare ID
CLIENT_SECRET = 'MIJZAJNLVUZ0GB1DWIEPGAK4XXJVQFFZFOUT2EN1DHA4E2D4' # your Foursquare Secret
ACCESS_TOKEN = 'VP2X2PUFBQYAVKTTZUEIUCB3QYSNE5BI3ZOZSGOXU2XQZSTE' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lon, 
    radius, 
    LIMIT)
url # display URL

Coordinates of Parkwoods: 43.7532586 -79.3296565


'https://api.foursquare.com/v2/venues/explore?&client_id=4JW2V1AXWJXREM12VKKJCM4B50FZ1I4TYRS2URMC1WMZRTNM&client_secret=MIJZAJNLVUZ0GB1DWIEPGAK4XXJVQFFZFOUT2EN1DHA4E2D4&v=20180604&ll=43.7532586,-79.3296565&radius=500&limit=30'

Send the url and get the result

In [68]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607206975b8470476ea19502'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab

In [69]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

then pass the cleaned JSON data in a pandas dataframe

In [71]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,TTC stop #8380,Bus Stop,43.752672,-79.326351
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


Now create a function to repeat the same process to all the neighborhoods in Old City of Toronto ...

In [78]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

... and call it

In [80]:
NY_venues = getNearbyVenues(NY_df['Neighborhood'], NY_df['Latitude'], NY_df['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Glencairn
Don Mills South
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


Let's see the dataframe

In [81]:
print(NY_venues.shape)
NY_venues.head()

(202, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Let's check how many venues were returned for each neighborhood

In [82]:
pd.DataFrame(NY_venues.groupby('Neighborhood')['Venue'].count())

,Venue
Neighborhood,
"Bathurst Manor, Wilson Heights, Downsview North",23
Bayview Village,4
"Bedford Park, Lawrence Manor East",24
Don Mills North,4
Don Mills South,20
Downsview Central,2
Downsview East,3
Downsview Northwest,5
Downsview West,5


Let's find out how many unique categories can be curated from all the returned venues

In [84]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 91 uniques categories.


Now analize each neighborhood

In [89]:
#one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']])
NY_onehot.columns = [col.split("_")[-1] for col in NY_onehot.columns]
# add neighborhood column back to dataframe
NY_onehot['Neighborhood'] = NY_venues['Neighborhood']

# move neighborhood column to the first column
index=NY_onehot.columns.get_loc('Neighborhood')
fixed_columns = [NY_onehot.columns[index]] + list(NY_onehot.columns[:index]) + list(NY_onehot.columns[index+1:])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Sporting Goods Shop,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


and verify data size

In [90]:
NY_onehot.shape

(202, 92)

Now we group the neighborhoods by the average of each category

In [91]:
NY_grouped=NY_onehot.groupby('Neighborhood').mean().reset_index()
NY_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Sporting Goods Shop,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.086957,0.000000,0.0,0.0,0.00,0.00,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.086957,0.000000,0.0,0.0,0.000000,0.043478,0.000000,0.00,0.043478,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.043478,0.000000,0.043478,0.000000,0.0,0.000000,0.043478,0.00,0.0,0.043478,0.0,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.043478,0.043478,0.000000,0.000000,0.043478,0.043478,0.0,0.043478,0.000000,0.0,0.0,0.000000,0.0,0.043478,0.000000,0.043478,0.043478,0.00,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.250000,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.041667,0.000000,0.00,0.000000,0.000000,0.083333,0.041667,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.041667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.041667,0.041667,0.00,0.0,0.000000,0.0,0.000000,0.041667,0.000000,0.083333,0.000000,0.041667,0.041667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.041667,0.0,0.041667,0.000000,0.0,0.0,0.041667,0.0,0.041667,0.000000,0.083333,0.000000,0.00,0.000000,0.000000,0.041667,0.041667,0.000000,0.041667,0.0,0.000000,0.000000
3,Don Mills North,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.0

and verify size

In [92]:
NY_grouped.shape

(23, 92)

Let's print each neighborhood along with the top 5 most common venues

In [94]:
num_top_venues = 5

for hood in NY_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NY_grouped[NY_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0               Bank  0.09
1        Coffee Shop  0.09
2           Pharmacy  0.04
3         Restaurant  0.04
4  Health Food Store  0.04


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4         Liquor Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.08
2      Sandwich Place  0.08
3       Grocery Store  0.04
4           Juice Bar  0.04


----Don Mills North----
                  venue  freq
0                   Gym  0.25
1  Caribbean Restaurant  0.25
2                  Café  0.25
3   Japanese Restaurant  0.25
4     Accessories Store  0.00


----Don Mills South----
                 venue  freq
0                  Gym  0.10
1           Restaurant  0.10
2          Coffee Shop  0.10
3       Clothing Store

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order

In [95]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [101]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
NY_venues_sorted = pd.DataFrame(columns=columns)
NY_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    NY_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

NY_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Bridal Shop,Pet Store,Pharmacy,Pizza Place,Mobile Phone Shop,Grocery Store,Ice Cream Shop
1,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pub,Liquor Store,Butcher,Indian Restaurant,Pet Store,Pharmacy
3,Don Mills North,Caribbean Restaurant,Gym,Japanese Restaurant,Café,Food & Drink Shop,Deli / Bodega,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant
4,Don Mills South,Coffee Shop,Gym,Restaurant,Grocery Store,Sandwich Place,Bike Shop,Beer Store,Dim Sum Restaurant,Chinese Restaurant,Italian Restaurant


Now we have to Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [104]:
# set number of clusters
kclusters = 5

NY_grouped_clustering = NY_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 0, 1, 1, 1, 2,
       3], dtype=int32)

In [105]:
# add clustering labels
NY_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NY_merged = NY_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
NY_merged = NY_merged.join(NY_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

NY_merged# check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Bus Stop,Food & Drink Shop,Department Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Deli / Bodega,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Sporting Goods Shop,Boutique,Coffee Shop,Gift Shop,Accessories Store,Cosmetics Shop,Caribbean Restaurant
7,M3B,North York,Don Mills North,43.745906,-79.352188,1.0,Caribbean Restaurant,Gym,Japanese Restaurant,Café,Food & Drink Shop,Deli / Bodega,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant
10,M6B,North York,Glencairn,43.709577,-79.445073,1.0,Bakery,Smoke Shop,Japanese Restaurant,Italian Restaurant,Vietnamese Restaurant,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
13,M3C,North York,Don Mills South,43.725900,-79.340923,1.0,Coffee Shop,Gym,Restaurant,Grocery Store,Sandwich Place,Bike Shop,Beer Store,Dim Sum Restaurant,Chinese Restaurant,Italian Restaurant
27,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Dog Run,Pool,Vietnamese Restaurant,Cosmetics Shop,Chocolate Shop,Clothing Store,Coffee Shop
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,1.0,Coffee Shop,Bank,Middle Eastern Restaurant,Bridal Shop,Pet Store,Pharmacy,Pizza Place,Mobile Phone Shop,Grocery Store,Ice Cream Shop
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0,Coffee Shop,Clothing Store,Bank,Juice Bar,Restaurant,Burger Joint,Electronics Store,Video Game Store,Department Store,Cosmetics Shop
34,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,1.0,Caribbean Restaurant,Bar,Metro Station,Coffee Shop,Massage Studio,Furniture / Home Store,Electronics Store,Dog Run,Chocolate Shop,Clothing Store


Finally, let's visualize the resulting clusters

In [109]:
# create map
map_clusters = folium.Map(location=loc_Toronto, zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], NY_merged['Neighborhood'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if cluster > -1:
        folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters